In [ ]:
%cd drive/MyDrive/Colab\ Notebooks/COMP8420

/content/drive/MyDrive/Colab Notebooks/COMP8420


# Import modules

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from pandas import DataFrame

In [ ]:
import os
from typing import Tuple
from tqdm.notebook import tqdm

# Load data function

In [ ]:
def load_data(path:str, mode:str)-> DataFrame:
    """This function load data from specific data file based on mode

    Args:
        path (str): path to folder contains data file
        mode (str): what data to be loaded

    Returns:
        DataFrame: DataFrame contains all data
    """
    assert mode in ["gsr", "pupil", "skin", "all"], f"mode should be 'gsr','pupil','all'. Not {mode}"
    # read data gsr
    if mode in ['gsr', 'all']:
        df_gsr = pd.read_excel(path+'gsr_features.xlsx',sheet_name='data')
        df_gsr.columns = ['patient'] + list(df_gsr.columns)[1:]
        if mode == 'gsr':
            return df_gsr
    # read pupil data
    if mode in ['pupil', 'all']:
        df_pupil = pd.read_excel(path+'pupil_features.xlsx',sheet_name='data')
        df_pupil.columns = ['patient'] + list(df_pupil.columns)[1:]
        if mode == 'pupil':
            return df_pupil
    # read skin temperature data
    if mode in ['skin', 'all']:
        df_skin = pd.read_excel(path+'skintemp_features.xlsx',sheet_name='data')
        df_skin.columns = ['patient'] + list(df_skin.columns)[1:]
        if mode == 'skin':
            return df_skin
    # in case of using all data, combine
    data = pd.concat([df_gsr,df_pupil,df_skin],axis = 1)
    # remove duplicated columns
    data = data.loc[:,~data.columns.duplicated()]
    return data

def train_val_test_split(data:DataFrame, test_patient:str, val_patient:str) -> Tuple[DataFrame,DataFrame,DataFrame]:
    """ This function is to create train, test and validation dataset using leave-one-patient-out

    Args:
        data (DataFrame): the dataset that contains all data point
        test_patient (str): patient choosen for testing
        val_patient (str): patient choosen for validation in training process

    Returns:
        Tuple[DataFrame,DataFrame,DataFrame]: train, validation, test dataframe
    """
    # check input
    assert test_patient in data.patient.unique(), f'{data.patient.unique().tolist()}'
    assert val_patient in data.patient.unique(), f'{data.patient.unique().tolist()}'
    # create train and test dataset
    df_test = data[data['patient'] == test_patient]
    df_train = data[(data['patient'] != test_patient)&((data['patient'] != val_patient))]
    df_val = data[data['patient'] == val_patient]
    # reset index
    df_test = df_test.reset_index(drop=True)
    df_train = df_train.reset_index(drop=True)
    df_val = df_val.reset_index(drop = True)
    # delete patient column
    if 'patient' in df_train.columns:
        del df_train['patient']
    if 'patient' in df_test.columns:
        del df_test['patient']  
    if 'patient' in df_val.columns:
        del df_val['patient']
    return df_train, df_val, df_test


def df_to_xy(df:DataFrame) -> Tuple[np.ndarray, np.ndarray]:
    """Convert dataframe to X and Y numpy array where X is input and Y is output

    Args:
        df (DataFrame): dataset to convert to numpy array

    Returns:
        Tuple[np.ndarray, np.ndarray]: X,Y
    """
    df = df.apply(pd.to_numeric)
    x = df.values[:,1:]
    y = df.values[:,0]
    return x,y

# Model

In [ ]:
class BaselineNN(nn.Module):
    """Baseline Neural Network with 1 hidden layer
    """
    def __init__(self, input_size: int, hidden_size:int, output_size:int, p = 0.7):
        """
        Args:
            input_size (int): input size of input vector
            hidden_size (int): size of hidden layer
            output_size (int): size of output layer
            p (float): probability of Dropout
        """
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        # from input to hidden layer
        self.in2hid = nn.Linear(self.input_size,self.hidden_size, bias=True)
        # from hidden to output layer
        self.hid2out = nn.Linear(self.hidden_size, self.output_size, bias=True)
        # dropout layer to generalize
        self.dropout = nn.Dropout(p)

    def forward(self, input):
        x = self.in2hid(input)
        x = torch.sigmoid(x)
        x = self.dropout(x)
        output = self.hid2out(x)
        return output

# Ultities

In [ ]:
def accuracy(Y_pred, Y_target):
    """ Calculate accuracy of prediction (Y_pred) compared to truth label (Y_target)
    Args:
        Y_pred (torch.tensor): prediction tensor with size of [batch x num_class]
        Y_target (torch.tensor): truth tensor with size of [batch x 1]
    Returns:
        torch.float: the overall accuracy
    """
    # find number of correct label
    correct_num = (Y_pred.argmax(-1) == Y_target).int().sum()
    # calculate accuracy
    acc = correct_num.float()/Y_target.shape[0]
    return acc


def recall(Y_pred, Y_target, class_label):
    """ Calculate Recall
    Recall = TP/(TP+FN)
    Args:
        Y_pred (torch.tensor): prediction tensor with size of [batch x num_class]
        Y_target (torch.tensor): truth tensor with size of [batch x 1]
        claass_label (str): label of class that we want to calculate recall
    Returns:
        torch.float: recall of model
    """
    # get label prediction
    Y_pred = Y_pred.argmax(-1)
    # find all datapoint that is true for the class_label
    label_index = torch.where(Y_target == class_label)
    # find all correctly predicted label in this class
    correct_label = (Y_pred[label_index] == Y_target[label_index]).int().sum()
    # returnr recall
    return float(correct_label)/len(label_index[0])


def precision(Y_pred, Y_target, class_label):
    """ Calculate Precision
    Recall = TP/(TP+FP)
    Args:
        Y_pred (torch.tensor): prediction tensor with size of [batch x num_class]
        Y_target (torch.tensor): truth tensor with size of [batch x 1]
        claass_label (str): label of class that we want to calculate precision
    Returns:
        torch.float: precision of model
    """
    # get label prediction
    Y_pred = Y_pred.argmax(-1)
    # get all correct label index
    label_index = torch.where(Y_target == class_label)
    # get all correctly predicted label for the class label
    correct_label = (Y_pred[label_index] == Y_target[label_index]).int().sum()
    # return precision
    return float(correct_label)/max(len(torch.where(Y_pred == class_label)[0]),1)


def f1(recall, precision):
    """ Calculate F1 score
    F1 =  2 * precision * recall/(precision + recall)
    Args:
        recall (float): recall value
        precision (float): precision value
    Returns:
        float: f1 score
    """
    return 2 * recall * precision / max((recall + precision),1)

In [ ]:
def train_model(X_train, Y_train, num_epoch, print_freq, dev):
    """
    This function is used to train baseline model
    Args:
        X_train (torch.tensor): input tensor used for training with size of [batch x dimension]
        Y_train (torch.tensor): output tensor used for training with size of [batch x 1]
        num_epoch (int): number of training epoch
        print_freq (int or bool): print process for debugging, if don't need to, set it to False
    Returns:
        Tuple(mode, loss_function): return trained model and loss function
    """
    # get input size from input variable
    input_size = X_train.shape[1]
    # output size
    output_size = 4
    # hidden layer size
    hidden_size = 50
    # create model
    model = BaselineNN(input_size, hidden_size, output_size)
    # model.to(dev)
    # create loss function
    criterion = nn.CrossEntropyLoss()
    # use Adam as optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    # for epoch in tqdm(range(num_epoch)):
    for epoch in range(num_epoch):
        # set model to train mode
        model.train()
        optimizer.zero_grad()

        outputs = model(X_train)
        train_loss = criterion(outputs, Y_train)
        # calculate accuracy if need to print out
        train_acc = (outputs.argmax(-1) == Y_train).int().sum()/X_train.shape[0]

        train_loss.backward()
        optimizer.step()
        # when print_preq != False
        if print_freq:
            if epoch % print_freq == 0:
                print(f'[EPOCH {epoch+1}/{num_epoch}] train_loss {train_loss.item():.4f}\t train_acc {train_acc:.4f}')
                # pass
    return model, criterion

def validation(model, criterion, X_val, Y_val, dev, test = False):
    """ Calculate all metrics of trained model
    Args:
        model (nn.Module): the model we need to evaluate
        criterion (nn.Module): loss function
        X_val (torch.tensor): input tensor used for validation with size of [batch x dimension]
        Y_val (torch.tensor): output tensor used for validation with size of [batch x 1]
        test (bool): set this to print output to debug/compare
    Returns:
        loss: loss function value
        acc: accuracy of model
        rc_all: recall for each class
        pr_all: precision for each class
        f1_all: F1 score for each class
    """
    # set model to evaluation mode
    model.eval()
    # X_val = X_val.to(dev)
    # Y_val = Y_val.to(dev)
    # calculate output
    outputs = model(X_val)
    # calculate loss_function
    loss = criterion(outputs, Y_val)
    # calculate accuracy
    acc = accuracy(outputs, Y_val)

    rc_all = []
    pr_all = []
    f1_all = []
    for i in range(4):
        # recall
        rc = recall(outputs, Y_val, i)
        # precision
        pr = precision(outputs, Y_val, i)
        # f1
        f1_value = f1(rc,pr)
        # Store result
        rc_all.append(rc)
        pr_all.append(pr)
        f1_all.append(f1_value)
    if test:
        print('Predict\t',outputs.argmax(-1))
        print('Test\t',Y_val)
    return loss, acc, rc_all, pr_all, f1_all

# GA + NN

In [ ]:
def crossover(p1, p2, crossover_rate = 0.8):
    """ Crossover operator for Genetic Algorithm """
    N = p1.shape[0]
    c1 = p1.clone()
    c2 = p2.clone()
    if random.random() <= crossover_rate:
        position = torch.randint(0,N,(1,))
        # cross over
        c1 = torch.cat((p1[:position],p2[position:]))
        c2 = torch.cat((p2[:position],p1[position:]))
    return c1, c2

In [ ]:
def mutation(bistring, mutation_rate):
    """ Mutation operator for Genetic Algorithm """
    for i in range(len(bistring)):
        if random.random() <= mutation_rate:
            bistring[i] = 1 - bistring[i]

In [ ]:
import random

In [ ]:
def selection(population,N,accs):
    """ Stochastic Universal sampling
        From the pseudocode: https://en.wikipedia.org/wiki/Stochastic_universal_sampling
    """
    F = sum(accs)
    P = F/N
    start = random.uniform(0,P)
    pointers = [start + i * P for i in range(N)]
    return roulet_wheel_selection(population, pointers,accs)

def roulet_wheel_selection(population, points, scores):
    """ Stochastic Universal sampling
        From the pseudocode: https://en.wikipedia.org/wiki/Stochastic_universal_sampling
    """
    keep = []
    for p in points:
        i = 0
        while sum(scores[:i]) < p:
            i = i+1
        if i >= len(population):
            i = i - len(population)
        keep.append(population[i])
    return keep

In [ ]:
def genetic_algorithm(data, num_pop, mutation_rate, crossover_rate, max_generation, input_size, train_model, num_epoch, dev):
    """ Genetic Algorithm process
    Args:
        data: Tuple[torch.tensor]
            tuple contains training data and validation data
        num_pop: int
            number of population
        mutation_rate: float
            mutation probability
        crossover_rate: float
            crossover probability
        max_generation: int
            total generation
        input_size: int
            number of input features
        train_model: Callable
            a function which return a trained model and a loss function
        num_epoch: int
            number of training epoch
        dev: torch.device
            device to use in training
    Returns:
        best: torch.tensor
            best bitstring found by genetic algorithm
        best_acc: float
            best fitness with respect to best bitstring or the best accuracy
    """
    # extract data
    X_train, Y_train, X_val, Y_val = data
    # generate the first generation
    population = [torch.ones((input_size,),dtype=torch.float).to(dev) for _ in range(num_pop)]
    # train to get first best result (the fitness)
    X_train_sub = X_train * population[0]
    X_val_sub = X_val * population[0]
    model, loss_func = train_model(X_train, Y_train, num_epoch, False, dev)
    best = population[0].clone()
    val_loss, best_acc, rc_all, pr_all, f1_all = validation(model, loss_func, X_val_sub, Y_val, dev)
    # run through each generation
    for gen in tqdm(range(max_generation)):
        # calculate fitness of all population
        accs = []
        for child in population:
            # find the fitness of each child
            X_train_sub = X_train * child
            X_val_sub = X_val * child
            model, loss_func = train_model(X_train, Y_train, num_epoch, False, dev)
            val_loss, acc, rc_all, pr_all, f1_all = validation(model, loss_func, X_val_sub, Y_val, dev)
            accs.append(acc)
        # check if there exist new best solution
        for i in range(num_pop):
            if accs[i] > best_acc:
                best = population[i].clone()
                best_acc = accs[i]
                print(f'At generation {gen}, found new best {best_acc}')
                # print(best)
        selected = selection(population, num_pop, accs)
        # create next gen
        children = []
        for i in range(0, num_pop, 2):
            p1,p2 = selected[i], selected[i+1]
            for c in crossover(p1,p2,crossover_rate):
                mutation(c,mutation_rate)
                children.append(c)
        population = children
    return best, best_acc

In [ ]:
import pickle

def save_bitstring(bitstring,filename):
    """ save bitstring for latter use"""
    with open(filename,'wb') as file:
        pickle.dump(bitstring,file)

def load_bitstring(filename):
    """ load bitstring from pickle object"""
    with open(filename,'rb') as file:
        bitstring = pickle.load(file)
    return bitstring


In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [ ]:
# 12321
torch.manual_seed(87651)
# 268
np.random.seed(268)
# load data set
dataset = load_data('./data/', 'all')
# get all the patient to perform leave one patient out
patient_list = dataset.patient.unique().tolist()
# genetic algorithm setup
crossover_rate = 0.8
mutation_rate = 1/85
num_pop = 100
input_size = 85
max_generation = 80
num_epoch = 200
# record data
all_bitstring = [] 
all_acc = []
all_recall = []
all_precision = []
all_f1 = []
for patient in patient_list:
    print(f'Choose patiet {patient} as testing patient')
    # get accuracy of best model
    best_accuracy = 0
    best_p_val = None
    for val_p in patient_list:
        if val_p == patient:
            continue
        print(f'Choose patient {val_p} as validation patient')
        # get train, val, test data
        train, val, test = train_val_test_split(dataset, patient,val_p)
        # create x,y
        train_x, train_y = df_to_xy(train)
        val_x, val_y = df_to_xy(val)
        # calculate normalize parameter
        mean_train_x = train_x.mean(axis=0)
        std_train_x = train_x.std(axis=0)
        # normalize data
        normalize_train_x = (train_x - mean_train_x)/std_train_x
        normalize_val_x = (val_x - mean_train_x)/std_train_x
        # convert to tensor
        X_train = torch.tensor(normalize_train_x, dtype=torch.float)
        Y_train = torch.tensor(train_y, dtype=torch.long)

        X_val = torch.tensor(normalize_val_x, dtype=torch.float)
        Y_val = torch.tensor(val_y, dtype=torch.long)
        # train model
        model, loss_func = train_model(X_train, Y_train, num_epoch,False, device)
        # validate model
        val_loss, val_acc, rc_all, pr_all, f1_all = validation(model, loss_func, X_val,Y_val, device)
        if val_acc > best_accuracy:
            best_accuracy = val_acc
            best_p_val = val_p
    # get train, val, test data
    train, val, test = train_val_test_split(dataset, patient,best_p_val)
    # create x,y
    train_x, train_y = df_to_xy(train)
    val_x, val_y = df_to_xy(val)
    test_x, test_y = df_to_xy(test)
    # calculate normalize parameter
    mean_train_x = train_x.mean(axis=0)
    std_train_x = train_x.std(axis=0)
    # normalize data
    normalize_test_x = (test_x - mean_train_x)/std_train_x
    # convert to tensor
    X_train = torch.tensor(normalize_train_x, dtype=torch.float)
    Y_train = torch.tensor(train_y, dtype=torch.long)

    X_val = torch.tensor(normalize_val_x, dtype=torch.float)
    Y_val = torch.tensor(val_y, dtype=torch.long)

    X_test = torch.tensor(normalize_test_x, dtype=torch.float)
    Y_test = torch.tensor(test_y, dtype=torch.long)
    # get the subset of feature
    data = (X_train, Y_train, X_val, Y_val)
    best_bitstring, best_acc = genetic_algorithm(data, num_pop, mutation_rate, crossover_rate, max_generation, input_size, train_model, num_epoch, device)
    X_train_sub = X_train * best_bitstring
    model, loss_func = train_model(X_train_sub, Y_train, 900, False, device)
    X_test_sub = X_test * best_bitstring
    test_loss, test_acc, rc_all, pr_all, f1_all = validation(model, loss_func, X_test_sub,Y_test, True)
    # save information
    all_bitstring.append(best_bitstring)
    all_acc.append(test_acc)
    all_recall.append(rc_all)
    all_precision.append(pr_all)
    all_f1.append(f1_all)

Choose patiet p02 as testing patient
Choose patient p03 as validation patient
Choose patient p04 as validation patient
Choose patient p06 as validation patient
Choose patient p07 as validation patient
Choose patient p08 as validation patient
Choose patient p09 as validation patient
Choose patient p10 as validation patient
Choose patient p11 as validation patient
Choose patient p12 as validation patient
Choose patient p13 as validation patient
Choose patient p14 as validation patient


At generation 10, found new best 0.3125
At generation 24, found new best 0.4375
At generation 33, found new best 0.5

Choose patiet p03 as testing patient
Choose patient p02 as validation patient
Choose patient p04 as validation patient
Choose patient p06 as validation patient
Choose patient p07 as validation patient
Choose patient p08 as validation patient
Choose patient p09 as validation patient
Choose patient p10 as validation patient
Choose patient p11 as validation patient
Choose patient p12 as validation patient
Choose patient p13 as validation patient
Choose patient p14 as validation patient


At generation 0, found new best 0.1875
At generation 0, found new best 0.25
At generation 14, found new best 0.3125
At generation 26, found new best 0.375

Choose patiet p04 as testing patient
Choose patient p02 as validation patient
Choose patient p03 as validation patient
Choose patient p06 as validation patient
Choose patient p07 as validation patient
Choose patient p08 as validation patient
Choose patient p09 as validation patient
Choose patient p10 as validation patient
Choose patient p11 as validation patient
Choose patient p12 as validation patient
Choose patient p13 as validation patient
Choose patient p14 as validation patient


At generation 0, found new best 0.3125
At generation 1, found new best 0.375
At generation 48, found new best 0.4375

Choose patiet p06 as testing patient
Choose patient p02 as validation patient
Choose patient p03 as validation patient
Choose patient p04 as validation patient
Choose patient p07 as validation patient
Choose patient p08 as validation patient
Choose patient p09 as validation patient
Choose patient p10 as validation patient
Choose patient p11 as validation patient
Choose patient p12 as validation patient
Choose patient p13 as validation patient
Choose patient p14 as validation patient


At generation 1, found new best 0.3125
At generation 2, found new best 0.375
At generation 7, found new best 0.4375
At generation 16, found new best 0.5625
At generation 27, found new best 0.625
At generation 50, found new best 0.6875
At generation 59, found new best 0.75

Choose patiet p07 as testing patient
Choose patient p02 as validation patient
Choose patient p03 as validation patient
Choose patient p04 as validation patient
Choose patient p06 as validation patient
Choose patient p08 as validation patient
Choose patient p09 as validation patient
Choose patient p10 as validation patient
Choose patient p11 as validation patient
Choose patient p12 as validation patient
Choose patient p13 as validation patient
Choose patient p14 as validation patient


...

At generation 0, found new best 0.3125
At generation 2, found new best 0.375
At generation 29, found new best 0.5

Choose patiet p11 as testing patient
Choose patient p02 as validation patient
Choose patient p03 as validation patient
Choose patient p04 as validation patient
Choose patient p06 as validation patient
Choose patient p07 as validation patient
Choose patient p08 as validation patient
Choose patient p09 as validation patient
Choose patient p10 as validation patient
Choose patient p12 as validation patient
Choose patient p13 as validation patient
Choose patient p14 as validation patient


At generation 0, found new best 0.3125
At generation 0, found new best 0.375
At generation 2, found new best 0.4375
At generation 38, found new best 0.5

Choose patiet p12 as testing patient
Choose patient p02 as validation patient
Choose patient p03 as validation patient
Choose patient p04 as validation patient
Choose patient p06 as validation patient
Choose patient p07 as validation patient
Choose patient p08 as validation patient
Choose patient p09 as validation patient
Choose patient p10 as validation patient
Choose patient p11 as validation patient
Choose patient p13 as validation patient
Choose patient p14 as validation patient


At generation 10, found new best 0.3125
At generation 21, found new best 0.375

Choose patiet p13 as testing patient
Choose patient p02 as validation patient
Choose patient p03 as validation patient
Choose patient p04 as validation patient
Choose patient p06 as validation patient
Choose patient p07 as validation patient
Choose patient p08 as validation patient
Choose patient p09 as validation patient
Choose patient p10 as validation patient
Choose patient p11 as validation patient
Choose patient p12 as validation patient
Choose patient p14 as validation patient


At generation 0, found new best 0.25
At generation 0, found new best 0.3125
At generation 1, found new best 0.375
At generation 14, found new best 0.4375
At generation 60, found new best 0.5

Choose patiet p14 as testing patient
Choose patient p02 as validation patient
Choose patient p03 as validation patient
Choose patient p04 as validation patient
Choose patient p06 as validation patient
Choose patient p07 as validation patient
Choose patient p08 as validation patient
Choose patient p09 as validation patient
Choose patient p10 as validation patient
Choose patient p11 as validation patient
Choose patient p12 as validation patient
Choose patient p13 as validation patient


At generation 0, found new best 0.25
At generation 1, found new best 0.3125
At generation 2, found new best 0.4375
At generation 15, found new best 0.5
At generation 28, found new best 0.625



In [ ]:
all_acc = np.array(all_acc)
all_recall = np.array(all_recall)
all_precision = np.array(all_precision)
all_f1 = np.array(all_f1)
print("==============================")
print('Average accuracy:',all_acc.mean())
print('Average Recall:',all_recall.mean(axis=0))
print('Average Precision:',all_recall.mean(axis=0))
print('Average F1:',all_f1.mean(axis=0))

Average accuracy: 0.22395833
Average Recall: [0.125      0.08333333 0.33333333 0.35416667]
Average Precision: [0.125      0.08333333 0.33333333 0.35416667]
Average F1: [0.12361111 0.06805556 0.22041997 0.21511243]


In [ ]:
all_bitstring = np.array([bitstring.numpy() for bitstring in all_bitstring])

In [ ]:
save_bitstring(all_bitstring,'best_model_ga/bitstrings.obj')

In [ ]:
all_bitstring.sum(axis=0)

array([ 8., 11.,  8.,  9.,  8.,  8., 10.,  9.,  8.,  6.,  8.,  6.,  6.,
        9.,  7.,  7.,  6.,  6.,  5.,  7.,  9.,  8., 10., 12.,  9., 10.,
       10.,  8., 11.,  9.,  7.,  8., 10.,  9.,  9.,  9.,  7., 10., 11.,
        8.,  5., 11.,  8.,  9.,  8.,  9.,  9.,  9.,  7.,  8.,  7.,  8.,
        9.,  7., 10.,  9., 10., 11.,  8.,  7.,  3.,  6.,  7., 10.,  8.,
        9.,  8.,  8., 10., 10., 11., 10.,  8.,  4.,  9., 11., 10.,  8.,
       10.,  9.,  8.,  8.,  6.,  8.,  6.], dtype=float32)